# MNIST with Tuning Neurons

This notebook is dedicated to study the effectof the tunning neurons in the MNIST task, the comparison should be done with other kind of ANNs already know in the industry.

**The main points to compare are**:
 - Performance (errors, accuracy, other metrics)
 - Learning Speed (in cycles)
 - Compare results of BOTH types of networks at initialization moment with a final Linear decoder
 - Operation complexity (number of OPS for example for each to get to a meaningful representation)
 - Compare a  random set of Tuning Neuron Ensembles (TNEs) with a final linear decoder
 - Create several ideas on attention mechanisms for the TNEs to generate different Deep TNEs Networks (DTNENs) and compare results

*NOTE*: The implementation of TNEs in TensorFlow might be needed for this task, as it might take a lot of time in the CPU.

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import random

from sklearn import linear_model
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

%matplotlib inline

In [2]:
from tensorflow.python import debug as tf_debug

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Tuning Neurons

In [4]:
#issue with TensorFlow, reference here: https://github.com/tensorflow/tensorflow/issues/6698
#config needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
# sess = tf_debug.LocalCLIDebugWrapperSession(sess)


### Definitions and neuron creation

In [5]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [6]:
# def create_tuning_neuron_2d(shape, min_y=0.5,
#                                max_y=1.5, min_weight=0.0, max_weight=0.8,
#                                min_x=-1.0, max_x=1.0, saturation=None,
#                               ):
#     height, width, input_channels, output_channels = shape #NHWC
#     n_neurons = n_synapses = width * height
#     # the first point is for y=0
#     nshape = [height, width] #, output_channels]
#     y1 = tf.zeros((n_neurons, ))
#     x1 = tf.random_uniform((n_neurons, ), -1, 1)
#     # the second point is for x = +-1
#     y2 = tf.random_uniform((n_neurons, ),min_y, max_y)
#     vec = np.random.choice([-1,1], (n_neurons, ))
#     x2 = tf.convert_to_tensor(vec, tf.float32) #will define a's sign
#     #a = tf.Variable( tf.divide(tf.subtract(y1,y2), tf.subtract(x1,x2)))
#     a = tf.divide(tf.subtract(y1,y2), tf.subtract(x1,x2))
#     A = a
#     #A = tf.stack([a for i in range(output_channels)])#, axis=1)
#     b = tf.subtract(y1, (a - x1))
#     B = b
#     #b = tf.Variable(tf.subtract(y1, (a - x1)))
#     #B = tf.stack([b for i in range(output_channels)])#, axis=1)
#     W = tf.Variable(tf.random_uniform([n_synapses,n_neurons]))
#     if(saturation is None):
#         saturation = [random.uniform(0.8, 1.0),1.0]
#     sat = tf.random_uniform((n_neurons, ), *saturation)
#     SAT = sat
#     return (A, B, SAT, W)

In [7]:
def create_tuning_neuron_2d(shape, min_y=0.5,
                               max_y=1.5, min_weight=0.0, max_weight=0.8,
                               min_x=-1.0, max_x=1.0, saturation=None,
                              ):
    height, width, input_channels, output_channels = shape #NHWC
    n_neurons = n_synapses = width * height
    # the first point is for y=0
    y1 = tf.zeros((n_neurons, ))
    #x1 = tf.random_uniform((n_neurons, ), -1, 1)
    x1 = tf.zeros((n_neurons, )) #try with the equivalent of a relu
    #tf.random_uniform((n_neurons, ), -1, 1)
    # the second point is for x = +-1
    #y2 = tf.random_uniform((n_neurons, ), min_y, max_y)
    #vec = np.random.choice([-1,1], (n_neurons, ))
    #x2 = tf.convert_to_tensor(vec, tf.float32) #will define a's sign
    #try with the equivalent of a relu
    y2 = tf.ones((n_neurons, ))
    x2 = tf.ones((n_neurons, ))
    #a = tf.Variable( tf.divide(tf.subtract(y1,y2), tf.subtract(x1,x2)))
    #a = tf.divide(tf.subtract(y1,y2), tf.subtract(x1,x2))
    a = tf.ones((n_neurons,))
    #a = tf.ones((height, width))
    #a = tf.ones((n_neurons, output_channels))
    A = a
    #A = tf.stack([a for i in range(output_channels)], axis=2)
    #b = tf.subtract(y1, (a - x1))
    b = tf.zeros((n_neurons, ))
    #b = tf.zeros((n_neurons, output_channels, ))
    #b = tf.Variable(tf.subtract(y1, (a - x1)))
    #B = tf.stack([b for i in range(output_channels)])#, axis=1)
    B = b
    #W = tf.Variable(tf.random_uniform([n_synapses,n_neurons]))
    W = tf.random_uniform([n_synapses,n_neurons])
    #W = tf.random_uniform([n_synapses,n_neurons, output_channels])
    #W = tf.random_uniform((n_synapses * n_neurons * output_channels, ))
                   
    if(saturation is None):
        saturation = [random.uniform(0.8, 1.0),1.0]
    sat = tf.random_uniform((n_neurons, ), *saturation)
    SAT = sat
    #SAT = tf.stack([sat for i in range(output_channels)])#, axis=1)
    #outputs placeholder
    return (A,B,SAT,W)

In [8]:
def evaluate_tuning_neuron(shape, A,B,SAT,W,x):
    height, width, input_channels, output_channels = shape #NHWC
    #regression logit model
    # I will first play with reshaping as I don't understand TF yet too much
    # it seems that reshaping is the ONLY available solution for tensorflow
    #xresh = tf.transpose(tf.reshape(x, [batch_size, height*width, output_channels]))
    #print(A.shape, B.shape, W.shape, SAT.shape, x.shape)
    xresh = tf.transpose(tf.reshape(x, [-1, height*width, output_channels ]),perm=[0,2,1])
    #xresh = tf.reshape(x, [-1, height * width, output_channels ])
    #Wr = tf.reshape(W, [-1, height*width, height*width])
    #try with another resizing and see what gives!!
    xresh2 = tf.reshape(x, [-1, height*width])
    xcurrent = tf.matmul(xresh2, W)
    #xcurrent = tf.matmul(xresh, W)
    #yb = tf.maximum(tf.minimum(tf.multiply(xcurrent, A) + B, sat), 0 )
    #print(A.shape, B.shape, W.shape, SAT.shape, x.shape, xcurrent.shape, xresh.shape)
    #yb = tf.maximum(tf.minimum(tf.multiply(xcurrent, A) + B, SAT), 0 )
    #yb = tf.maximum(tf.multiply(xcurrent, A) + B, 0 )
    #yb = tf.transpose(tf.reshape(xcurrent, [-1, output_channels, height, width]), perm=[0,1,3,2])
    yb = tf.maximum(tf.matmul(xcurrent, tf.reshape(A,(height*width,-1))), 0 )
    #y = tf.maximum(tf.multiply(yb, A), 0 )
    #yb = tf.maximum( x, 0)
    y = tf.transpose(tf.reshape(yb, [-1, output_channels, height, width]), perm=[0,2,3,1])
    #try with a relu!
    #y = tf.maximum( tf.multiply(x, A), 0)
    #y = tf.maximum( x, 0)
    return y

In [9]:
tx = tf.placeholder(tf.float32, shape=[None, 784])
ty_ = tf.placeholder(tf.float32, shape=[None, 10])
batch_size = 100

In [10]:
#patch [width, height, input channels, output channels]
tW_conv1 = weight_variable([5, 5, 1, 32])
tb_conv1 = bias_variable([32]) #number of output bias
#4d tensor [ N# batch inputs, width, height, n#channels]
tx_image = tf.reshape(tx, [-1, 28, 28, 1])

thconv1 = conv2d(tx_image, tW_conv1) + tb_conv1
#thconv1 = tf.Print(thconv1, [thconv1.shape])
shape1 = [28, 28, 1, 32 ]
A1,B1,SAT1,W1 = create_tuning_neuron_2d(shape1, saturation=[0.8,10])

In [11]:
A1

<tf.Tensor 'ones_2:0' shape=(784,) dtype=float32>

In [12]:
th_conv1 = evaluate_tuning_neuron(shape1, A1,B1,SAT1,W1, thconv1)
# th_conv1 = tf.Print(th_conv1, [th_conv1.shape])
th_pool1 = max_pool_2x2(th_conv1)

# th_conv1 = tf.nn.relu(conv2d(tx_image, tW_conv1) + tb_conv1)
# th_pool1 = max_pool_2x2(th_conv1)
# th_pool1 = tf.Print(th_pool1, [th_pool1.shape])

In [13]:
th_conv1

<tf.Tensor 'transpose_1:0' shape=(?, 28, 28, 32) dtype=float32>

In [22]:
th_pool1

<tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>

In [14]:
tW_conv2 = weight_variable([5, 5, 32, 64])
tb_conv2 = bias_variable([64])

# thconv2 = conv2d(th_pool1, tW_conv2) + tb_conv2
# (a, b, sat, W, x, th_conv2, y_ )= create_tuning_neuron_2d([14, 14, 1, 64], thconv2,
#                                                    saturation=[0.8,10])
# th_pool2 = max_pool_2x2(th_conv2)

# shape2 = [14, 14, 1, 64 ]
# A2,B2,SAT2,W2 = create_tuning_neuron_2d(shape2, saturation=[0.8,10])
# th_conv2 = evaluate_tuning_neuron(shape2, A2,B2,SAT2,W2, thconv2)
# th_conv2 = tf.Print(th_conv1, [th_conv1.shape])
# th_pool2 = max_pool_2x2(th_conv2)

th_conv2 = tf.nn.relu(conv2d(th_pool1, tW_conv2) + tb_conv2)
th_pool2 = max_pool_2x2(th_conv2)


In [15]:
th_pool2

<tf.Tensor 'MaxPool_1:0' shape=(?, 7, 7, 64) dtype=float32>

In [16]:
#Densely connected layer
tW_fc1 = weight_variable([7 * 7 * 64, 1024])
tb_fc1 = bias_variable([1024])

th_pool2_flat = tf.reshape(th_pool2, [-1, 7*7*64])

In [23]:
th_pool2_flat

<tf.Tensor 'Reshape_5:0' shape=(?, 3136) dtype=float32>

In [17]:
#is HERE that I have to replace by my Tuning Neurons

# thconv1 = conv2d(tx_image, tW_conv1) + tb_conv1
# (a, b, sat, W, x, th_conv1, y_ )= create_tuning_neuron_2d([28, 28, 1, 32 ], 
#                                                           thconv1, saturation=[0.8,10])
# th_pool1 = max_pool_2x2(th_conv1)

th_fc1 = tf.nn.relu(tf.matmul(th_pool2_flat, tW_fc1) + tb_fc1)

#Dropout

tkeep_prob = tf.placeholder(tf.float32)
th_fc1_drop = tf.nn.dropout(th_fc1, tkeep_prob)

#Readout Layer

tW_fc2 = weight_variable([1024, 10])
tb_fc2 = bias_variable([10])

ty_conv = tf.matmul(th_fc1_drop, tW_fc2) + tb_fc2

In [18]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=ty_, logits=ty_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(ty_conv, 1), tf.argmax(ty_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [19]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=ty_, logits=ty_conv))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# sess.run(tf.global_variables_initializer())

# for _ in range(1000):
#   batch = mnist.train.next_batch(batch_size)
#   train_step.run(feed_dict={tx: batch[0], ty_: batch[1]})

In [20]:
%%time 

sess.run(tf.global_variables_initializer())
for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i>0 and i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          tx: batch[0], ty_: batch[1], tkeep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
      if(train_accuracy > 0.999):
        break
    train_step.run(feed_dict={tx: batch[0], ty_: batch[1], tkeep_prob: 0.5})

InvalidArgumentError: Input to reshape is a tensor with 1600 values, but the requested shape requires a multiple of 25088
	 [[Node: Reshape_4 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/gpu:0"](Maximum, Reshape_4/shape)]]

Caused by op 'Reshape_4', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/zmq/eventloop/ioloop.py

Results seem completely random, as if it was creating a NEW encoding each time ???

I really don't understand it

In [21]:

print('test accuracy %g' % accuracy.eval(feed_dict={
x: mnist.test.images, y_: mnist.test.labels, tkeep_prob: 1.0}))


NameError: name 'x' is not defined

Still with issues about the dimensions:

    InvalidArgumentError: In[0].dim(0) and In[1].dim(0) must be the same: [10000,32,784] vs [1,784,784]
         [[Node: MatMul_3 = BatchMatMul[T=DT_FLOAT, adj_x=false, adj_y=false, _device="/job:localhost/replica:0/task:0/gpu:0"](transpose, Reshape_9)]]
         [[Node: Mean_3/_7 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_239_Mean_3", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

    During handling of the above exception, another exception occurred:
    
After this I must check on HOW to do the correct things for the trainings...

In [ ]:
vals = sess.run(tf.trainable_variables())

In [ ]:
for k,v in zip([v.name for v in tf.trainable_variables()], vals):
    print ("var: "+k+" = "+str(v.shape))